In [ ]:
# Author: Chelsea Ekstrand <chelsea.ekstrand@uleth.ca>

In [2]:
# import functions

import numpy as np
import scipy
import mne as mne
import nibabel as nib
from mne.stats import spatio_temporal_cluster_1samp_test, spatio_temporal_cluster_test
from os import path

In [21]:
# set variables

cond1data = '/media/ekstrand/BackupPlus2/NaturalisticDataset_v2/sub_face_averages/20sub_average_task-500daysofsummer_cond-face_TRtimepoints-11.nii.gz' # path to restructured data file for condition 1
cond2data = '/media/ekstrand/BackupPlus2/NaturalisticDataset_v2/sub_noface_averages/20sub_average_task-500daysofsummer_cond-noface_TRtimepoints-11.nii.gz'# path to restructured data file for condition 1
cond1name = 'face' # name of condition 1
cond2name = 'noface' # name of condition 2
saveDir = '/media/ekstrand/BackupPlus2/NaturalisticDataset_v2/sub_face_averages/'# path to save results
pval = .001 
n_permutations = 5000
test = 'paired' # type of t-test to perform, options: 'ind' = independent, 'paired' = paired
tail = 0 # 0 = two-tailed test, 1 = one-tailed test, thresholded above 0, -1 = one-tailed test, thresholded below 0

In [22]:
# perform desired t-test and save results

def save_nifti(filename,affine,data):
    img = nib.Nifti1Image(data,affine)
    nib.save(img, filename)
    return()

# load functional data
print('Loading fMRI data for condition 1: ' + cond1name)
load_c1_data = nib.load(cond1data)
c1_data = load_c1_data.get_fdata()
print('Loading fMRI data for condition 2: ' + cond2name)
load_c2_data = nib.load(cond2data)
c2_data = load_c2_data.get_fdata()

N1,t,x,y,z = np.shape(c1_data)
N2,t,x,y,z = np.shape(c2_data)

#create a regular lattice adjacency for each spatial dimension
x_adjacency,y_adjacency,z_adjacency,t_adjacency=(x,y,z,t)
adjacency = mne.stats.combine_adjacency(x_adjacency, y_adjacency, z_adjacency)

if test == 'paired':
    print('Running paired samples t-test for ' + str(N1) + ' participants')
    
    df = N1 - 1
    if tail == 0:
        thresh =  scipy.stats.t.ppf(1-pval/2,df)
    elif tail == 1:
        thresh =  scipy.stats.t.ppf(1-pval,df)
    elif tail == -1:
        thresh =  scipy.stats.t.ppf(pval,df)
    
    contrast = c1_data - c2_data
    
    # perform t-test
    t_obs,clusters,cluster_pv,H0 = clu = mne.stats.spatio_temporal_cluster_1samp_test(
        X=contrast,
        stat_fun=None,
        adjacency=adjacency,
        threshold=thresh,
        tail=tail,
        n_permutations=n_permutations,
        out_type='mask',
        max_step=1)
    
    # save output
    print('Saving results to ' + saveDir)
    contrast_ave = contrast.mean(axis=0)
    signs = np.sign(contrast_ave)
    
    T_obs_plot = np.nan * np.ones_like(t_obs)
    clust_fill = np.nan * np.ones_like(t_obs)
    
    # mask t-statistic images with significant clusters
    i=1
    for c, p_val in zip(clusters, cluster_pv):
        if p_val <= 0.05:
            T_obs_plot[c] = t_obs[c]
            clust_fill[c] = signs[c]*i
            i=i+1
    T_obs_plot = np.transpose(T_obs_plot,(1,2,3,0))
    clust_fill = np.transpose(clust_fill,(1,2,3,0))
    contrast_ave = np.transpose(contrast_ave,(1,2,3,0))
    
    # save results
    affine_mat = load_c1_data.affine
    t_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_pairedt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_tstat.nii.gz')
    clust_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_pairedt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_clusters.nii.gz')
    ave_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_pairedt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_contrast_average.nii.gz')
    
    save_nifti(t_file,affine_mat,T_obs_plot)
    save_nifti(clust_file,affine_mat,clust_fill)
    save_nifti(ave_file,affine_mat,contrast_ave)
    
    print('Done!')
    
elif test == 'ind':
    print('Running independent samples t-test for ' + str(N1+N2) + ' participants')
    dfn = 2 - 1  # degrees of freedom numerator
    dfd = N1 + N2 - 2  # degrees of freedom denominator
    thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
    
    # perform t-test
    F_obs, clusters, cluster_pv, H0 = clu = mne.stats.spatio_temporal_cluster_test(
        X=[c1_data, c2_data],
        adjacency=adjacency,
        threshold=thresh,
        tail=tail,
        n_permutations=n_permutations,
        out_type='mask',
        max_step=1)
    
    c1_average = c1_data.mean(axis=0)
    c2_average = c2_data.mean(axis=0)
    
    c1_c2_contrast = c1_average-c2_average
    signs = np.sign(c1_c2_contrast)
    
    F_obs_plot = np.nan * np.ones_like(F_obs)
    clust_fill = np.nan * np.ones_like(F_obs)
    
    i=1
    for c, p_val in zip(clusters, cluster_pv):
        if p_val <= 0.05:
            F_obs_plot[c] = F_obs[c]*signs[c]
            clust_fill[c] = signs[c]*i
            i=i+1
            
    print('Saving results to ' + saveDir)
    F_obs_plot = np.transpose(F_obs_plot,(1,2,3,0))
    clust_fill = np.transpose(clust_fill,(1,2,3,0))
    c1_c2_contrast = np.transpose(c1_c2_contrast,(1,2,3,0))
    
    # save results
    affine_mat = load_c1_data.affine
    F_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_indt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_tstat.nii.gz')
    clust_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_indt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_clusters.nii.gz')
    contrast_file = path.join(saveDir,'4d_spatiotemporal_cluster_results_indt_' + cond1name + '_vs_' + cond2name + '_n_perm-' + str(n_permutations) + '_contrast_average.nii.gz')
    
    save_nifti(F_file,affine_mat,F_obs_plot)
    save_nifti(clust_file,affine_mat,clust_fill)
    save_nifti(contrast_file,affine_mat,c1_c2_contrast)
    
    print('Done!')

Loading fMRI data for condition 1: face
Loading fMRI data for condition 2: noface
Running independent samples t-test for 40 participants
stat_fun(H1): min=nan max=nan
Running initial clustering
Found 1249 clusters


/tmp/ipykernel_558633/1701037415.py:85: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  F_obs, clusters, cluster_pv, H0 = clu = mne.stats.spatio_temporal_cluster_test(


Permuting 49 times...


  0%|          |  : 0/49 [00:00<?,       ?it/s]

Computing cluster p-values
Done.
Saving results to /media/ekstrand/BackupPlus2/NaturalisticDataset_v2/sub_face_averages/
Done!
